In [1]:
## import the tensorflow APIs
import tensorflow as tf
import pandas as pd
import math
import random
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np

In [2]:
data_sitcoms = pd.read_csv("mustard++_text.csv")

# Adjust sitcom data
data_sitcoms = data_sitcoms.drop(columns=['SCENE','KEY','END_TIME','SPEAKER','SHOW','Sarcasm_Type','Implicit_Emotion','Explicit_Emotion','Valence','Arousal'], axis=1)
data_sitcoms = data_sitcoms.rename(columns={'SENTENCE':'text','Sarcasm':'label'})

# remove empty label rows
for index, row in data_sitcoms.iterrows():
    if math.isnan(row['label']):
        data_sitcoms = data_sitcoms.drop(index, axis='index')

data_sitcoms.head()

,text,label
5,"And of those few months, how long have you bee...",0.0
14,"Let the dead man talk. So, why do you think that?",0.0
18,"What else? Sell it on eBay as ""slightly used.""",0.0
24,"Good idea, sit with her. Hold her, comfort her...",1.0
31,"Well, now that I've given up string theory, I'...",0.0


In [3]:
train_sentences = data_sitcoms["text"]
##train_sentences

In [4]:
##instantiate the tokenizer
tokenizer = Tokenizer(num_words=100)

##train the tokenizer on training sentences
tokenizer.fit_on_texts(train_sentences)

##store word index for the words in the sentence
word_index = tokenizer.word_index
##print(word_index)

In [5]:
##create sequences using tokenizer
sequences = tokenizer.texts_to_sequences(train_sentences)

In [6]:
##print word index dictionary and sequences
##print(f"Word index -->{word_index}")
##print(f"Sequences of words -->{sequences}")

In [7]:
##print sample sentence and sequence
train_sentences = train_sentences.to_numpy()

In [8]:
print(train_sentences[random.randint(0,100)])
print(sequences[0])

No. The dark crescent-shaped patterns under your arms conceal it nicely.
[6, 8, 50, 24, 1, 4]


In [9]:
##set up the tokenizer again with oov_token
tokenizer = Tokenizer(num_words=100, oov_token = "<oov>")

##train the new tokenizer on training sentences
tokenizer.fit_on_texts(train_sentences)

##store word index for the words in the sentence
word_index = tokenizer.word_index

In [10]:
##pad sequences
padded_seqs = pad_sequences(sequences)

In [11]:
##print(word_index)
##print(train_sentences)
##print(sequences)
##print(padded_seqs)

In [12]:
##pad sequences with padding type, max length and truncating parameters
padded_seqs = pad_sequences(sequences,
                            padding="post",
                            maxlen=5,
                            truncating="post",
                            )
print(padded_seqs)

[[ 6  8 50 24  1]
 [ 3 29 59 27  1]
 [23  7 17 58  0]
 ...
 [31 72 89 67 42]
 [ 2  9 16  3  0]
 [ 9 42 57  0  0]]


In [13]:
##create empty list to store sentences and labels
train_sentences = []
test_sentences = []

train_labels = []
test_labels = []

In [14]:
#create size variables to for training
subset_size = len(data_sitcoms.index)
testing_size = int(subset_size * 0.2)
validation_size = testing_size
shuffle_size = subset_size - validation_size

In [15]:
data = data_sitcoms.sample(frac=1).reset_index(drop=True)
train_data = data.head(subset_size - testing_size)
test_data = data.tail(testing_size)

In [16]:
for sentence in train_data['text']:
    train_sentences.append(sentence)
for sentence in test_data['text']:
    test_sentences.append(sentence)

In [17]:
for value in train_data['label']:
    train_labels.append(value)
for value in test_data['label']:
    test_labels.append(value)

In [18]:
##convert lists into numpy array
train_labels = np.array(train_labels)
test_labels = np.array(test_labels)

In [19]:
##define the parameters for the tokenizing and padding
vocab_size = 10000
embedding_dim = 16 ##originally 16
max_length = 120
trunc_type='post'
oov_tok = "<OOV>"

In [20]:
tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

##training sequences and labels
train_seqs = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_seqs,maxlen=max_length, truncating=trunc_type)

##testing sequences and labels
test_seqs = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_seqs,maxlen=max_length)

In [21]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

print(train_sentences[1])
print(train_padded[1])
print(decode_review(train_padded[1]))

Imagine this: you've got the baby in one hand, groceries in the other, and you're thinking, "How am I gonna open this trunk?"
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
   0   0   0   0   0   0   0 522  27 205  66   4 230  12  67 324 855  12
   4  89   7  43 231  52  58   3  78 267  27 523]
? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? imagine this you've got the baby in one hand groceries in the other and you're thinking how am i gonna open this trunk


Define the Neural Network with Embedding layer

In [22]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

##compile the model with loss function, optimizer and metrics
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 16)           160000    
                                                                 
 flatten (Flatten)           (None, 1920)              0         
                                                                 
 dense (Dense)               (None, 6)                 11526     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


Model Training

In [23]:
num_epochs = 100

##train the model with training and validation set
model.fit(
    train_padded, 
    train_labels, 
    epochs=num_epochs, 
    validation_data=(test_padded, test_labels)
    )

Epoch 1/100
31/31 [==============================] - 1s 10ms/step - loss: 0.6946 - accuracy: 0.4917 - val_loss: 0.6930 - val_accuracy: 0.5542
Epoch 2/100
31/31 [==============================] - 0s 5ms/step - loss: 0.6921 - accuracy: 0.5530 - val_loss: 0.6922 - val_accuracy: 0.5083
Epoch 3/100
31/31 [==============================] - 0s 5ms/step - loss: 0.6894 - accuracy: 0.5800 - val_loss: 0.6918 - val_accuracy: 0.5125
Epoch 4/100
31/31 [==============================] - 0s 5ms/step - loss: 0.6834 - accuracy: 0.6445 - val_loss: 0.6903 - val_accuracy: 0.5042
Epoch 5/100
31/31 [==============================] - 0s 5ms/step - loss: 0.6704 - accuracy: 0.6694 - val_loss: 0.6923 - val_accuracy: 0.5083
Epoch 6/100
31/31 [==============================] - 0s 5ms/step - loss: 0.6477 - accuracy: 0.6580 - val_loss: 0.6872 - val_accuracy: 0.5083
Epoch 7/100
31/31 [==============================] - 0s 5ms/step - loss: 0.6051 - accuracy: 0.7058 - val_loss: 0.6860 - val_accuracy: 0.5208
Epoch 8/100


Derive weights from the embedding layer

In [24]:
##isolating the first embedding layer
l1 = model.layers[0]

##extracting learned weights
weights = l1.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)
print(weights[0])

(10000, 16)
[-0.07670038  0.0088451   0.06071059 -0.00197269  0.00648356  0.03113407
 -0.08285975  0.12546222 -0.06653283  0.06121429  0.02034753 -0.01793756
 -0.04366457 -0.00665691  0.0228914  -0.06364395]


Download the vectors and metadata

In [25]:
##import I/O module in python
import io

##open the text stream for vectors
vectors = io.open('vectors.tsv', 'w', encoding='utf-8')

##open the text stream for metadata
meta = io.open('meta.tsv', 'w', encoding='utf-8')

##write each word and its corresponding embedding
for index in range(len(reverse_word_index)):
  word = reverse_word_index.get(index)  # flipping the key-value in word_index
  embeddings = weights[index]
  if word is None:
    break
  else:
    meta.write(word + "\n")
    vectors.write('\t'.join([str(x) for x in embeddings]) + "\n")

##close the stream
vectors.close()
meta.close()

In [26]:
##download the written files to your local machine
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vectors.tsv')
  files.download('meta.tsv')